In [54]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

In [55]:
batch_size =64
epochs = 100
latent_dim = 256 # latent dimention for the encoding space
num_samples = 10000
data_path = 'fra.txt'

In [56]:
#Vectorize the data

input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding = 'utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) -1)]:
    input_text,target_text,_ = line.split('\t')
    target_text = '\t'+target_text+'\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [57]:
input_characters = sorted(list(input_characters))

In [58]:
target_characters = sorted(list(target_characters))

In [59]:
num_encoder_tokens = len(input_characters)

In [60]:
num_decoder_tokens = len(target_characters)

In [61]:
max_encoder_Seq_length = max([len(txt) for txt in input_texts])

In [62]:
max_decoder_Seq_length = max([len(txt) for txt in target_texts])

In [63]:
input_token_index = {char:i for (i, char) in enumerate(input_characters)}
target_token_index = {char:i for (i, char) in enumerate(target_characters)}

In [64]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_Seq_length, num_encoder_tokens), dtype = 'float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_Seq_length, num_decoder_tokens), dtype = 'float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_Seq_length, num_decoder_tokens), dtype = 'float32')

In [65]:
target_text

"\tJe peux t'entendre.\n"

In [66]:
for i, (input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1
    encoder_input_data[i, t+1:, input_token_index[' ']] = 1
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1
        if t>0:
            decoder_target_data[i, t-1, target_token_index[char]] = 1
    decoder_input_data[i, t+1:, target_token_index[' ']] = 1
    decoder_target_data[i, t:, target_token_index[' ']] = 1

In [67]:
#Defining the input sequence

encoder_inputs = Input(shape = (None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state = True)
encoder_output, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [68]:
decoder_inputs = Input(shape = (None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences = True, return_state = True)
decoder_outputs,_,_ = decoder_lstm(decoder_inputs, initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [69]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = batch_size, epochs = epochs,
          validation_split = 0.2)

Epoch 1/100
125/125 [==============================] - 26s 181ms/step - loss: 1.4691 - accuracy: 0.7075 - val_loss: 1.0063 - val_accuracy: 0.7206
Epoch 2/100
125/125 [==============================] - 24s 192ms/step - loss: 0.8513 - accuracy: 0.7703 - val_loss: 0.8072 - val_accuracy: 0.7729
Epoch 3/100
125/125 [==============================] - 25s 201ms/step - loss: 0.6997 - accuracy: 0.8102 - val_loss: 0.7169 - val_accuracy: 0.7958
Epoch 4/100
125/125 [==============================] - 26s 210ms/step - loss: 0.5820 - accuracy: 0.8304 - val_loss: 0.6328 - val_accuracy: 0.8167
Epoch 5/100
125/125 [==============================] - 25s 199ms/step - loss: 0.5327 - accuracy: 0.8440 - val_loss: 0.5817 - val_accuracy: 0.8317
Epoch 6/100
125/125 [==============================] - 25s 198ms/step - loss: 0.4950 - accuracy: 0.8545 - val_loss: 0.5539 - val_accuracy: 0.8392
Epoch 7/100
125/125 [==============================] - 25s 198ms/step - loss: 0.4680 - accuracy: 0.8611 - val_loss: 0.5317 -

125/125 [==============================] - 25s 200ms/step - loss: 0.0984 - accuracy: 0.9687 - val_loss: 0.5900 - val_accuracy: 0.8718
Epoch 57/100
125/125 [==============================] - 25s 200ms/step - loss: 0.0963 - accuracy: 0.9701 - val_loss: 0.5937 - val_accuracy: 0.8726
Epoch 58/100
125/125 [==============================] - 25s 199ms/step - loss: 0.0942 - accuracy: 0.9705 - val_loss: 0.5944 - val_accuracy: 0.8731
Epoch 59/100
125/125 [==============================] - 25s 197ms/step - loss: 0.0921 - accuracy: 0.9708 - val_loss: 0.6048 - val_accuracy: 0.8727
Epoch 60/100
125/125 [==============================] - 25s 202ms/step - loss: 0.0903 - accuracy: 0.9710 - val_loss: 0.6036 - val_accuracy: 0.8721
Epoch 61/100
125/125 [==============================] - 25s 198ms/step - loss: 0.0882 - accuracy: 0.9720 - val_loss: 0.6096 - val_accuracy: 0.8726
Epoch 62/100
125/125 [==============================] - 25s 199ms/step - loss: 0.0861 - accuracy: 0.9726 - val_loss: 0.6147 - val_a

In [70]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape = (latent_dim,))
decoder_state_input_c= Input(shape = (latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state = decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs]+decoder_states_inputs, [decoder_outputs]+decoder_states)
reverse_input_char_index = dict((i, char) for char,i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char,i in target_token_index.items())

def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1,1,num_decoder_tokens))
    target_seq[0,0,target_token_index['\t']] = 1
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens,h,c = decoder_model.predict([target_seq]+states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        if(sampled_char == '\n' or len(decoded_sentence)>max_decoder_Seq_length):
            stop_condition = True
        target_seq = np.zeros((1,1, num_decoder_tokens))
        target_seq[0,0, sampled_token_index] = 1
        states_value = [h,c]
    return decoded_sentence

for seq_index in range(100):
    input_seq = encoder_input_data[seq_index: seq_index+1]
    decoded_sentence = decode_sequence(input_seq)
    print('_')
    print('Input Sentence:', input_texts[seq_index])
    print('Decoded Sentence:', decoded_sentence)


_
Input Sentence: Go.
Decoded Sentence: Marche.

_
Input Sentence: Go.
Decoded Sentence: Marche.

_
Input Sentence: Go.
Decoded Sentence: Marche.

_
Input Sentence: Hi.
Decoded Sentence: Salut !

_
Input Sentence: Hi.
Decoded Sentence: Salut !

_
Input Sentence: Run!
Decoded Sentence: File !

_
Input Sentence: Run!
Decoded Sentence: File !

_
Input Sentence: Run!
Decoded Sentence: File !

_
Input Sentence: Run!
Decoded Sentence: File !

_
Input Sentence: Run!
Decoded Sentence: File !

_
Input Sentence: Run!
Decoded Sentence: File !

_
Input Sentence: Run!
Decoded Sentence: File !

_
Input Sentence: Run!
Decoded Sentence: File !

_
Input Sentence: Run.
Decoded Sentence: File !

_
Input Sentence: Run.
Decoded Sentence: File !

_
Input Sentence: Run.
Decoded Sentence: File !

_
Input Sentence: Run.
Decoded Sentence: File !

_
Input Sentence: Run.
Decoded Sentence: File !

_
Input Sentence: Run.
Decoded Sentence: File !

_
Input Sentence: Run.
Decoded Sentence: File !

_
Input Sentence: Ru